In [ ]:
!pip install --upgrade pip wheel

In [ ]:
!pip install requests tqdm faiss-cpu torch sentence-transformers textblob gensim numba ninja 'numpy<2' 'transformers==4.44.2' 'accelerate==0.33.0'

In [ ]:
!MAX_JOBS=12 python -m pip -v install flash-attn --no-build-isolation  --use-pep517

In [ ]:
import os
import requests
import zipfile
from pathlib import Path
from tqdm import tqdm
import re
import json
from gensim.utils import simple_preprocess
from textblob import TextBlob

# Directory to store downloaded and extracted data
DATA_DIR = Path("./mimic_textbooks")
# URLs for the dataset and pre-chunked JSON
dataset_url = "https://www.dropbox.com/scl/fi/gk1y8ll3d7wllwbb24kqe/textbooks.zip?rlkey=cdpqf8cbeu3difouvhwsc866w&st=resv96io&dl=1"
CHUNKED_DOCUMENTS_PATH = Path("./chunked_documents.json")
CHUNKED_DOCUMENTS_URL = "https://www.dropbox.com/scl/fi/07wd0zwvz2xcq80hy5f91/chunked_documents.json?rlkey=jwvfpczo4zeyke9j74cdphovi&st=oeqmcfi8&dl=1"

# Download and extract the dataset zip file
def download_and_extract_zip(url, extract_to=DATA_DIR):
    # Ensure the directory exists
    extract_to.mkdir(parents=True, exist_ok=True)

    # Download the zip file
    zip_path = extract_to / "textbooks.zip"
    print("Downloading dataset...")
    response = requests.get(url, stream=True)
    with open(zip_path, "wb") as file:
        for chunk in tqdm(response.iter_content(chunk_size=1024), unit='KB'):
            if chunk:
                file.write(chunk)

    # Extract the zip file
    print("Extracting dataset...")
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
    print("Dataset downloaded and extracted.")

# Load text files from a given directory
def load_text_files(directory):
    texts = []
    for file_path in Path(directory).glob("*.txt"):
        with open(file_path, "r", encoding="utf-8") as file:
            texts.append(file.read())
    return texts

# Cleaning and preprocessing function
def clean_and_tokenize(text):
    # Remove extra spaces, lowercase text, and remove special characters
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = simple_preprocess(text)
    return ' '.join(tokens)

# Chunk text into fixed-size chunks
def chunk_text(text, chunk_size=1000):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Main process:
if CHUNKED_DOCUMENTS_PATH.exists():
    print("Loading existing chunked_documents.json...")
    with open(CHUNKED_DOCUMENTS_PATH, "r", encoding="utf-8") as f:
        chunked_documents = json.load(f)
else:
    print("chunked_documents.json does not exist. Trying to download from remote URL...")
    try:
        response = requests.get(CHUNKED_DOCUMENTS_URL, allow_redirects=True)
        with open(CHUNKED_DOCUMENTS_PATH, "wb") as f:
            f.write(response.content)
        print("Successfully downloaded chunked_documents.json from remote URL.")
        with open(CHUNKED_DOCUMENTS_PATH, "r", encoding="utf-8") as f:
            chunked_documents = json.load(f)
    except Exception as e:
        print(f"Failed to download chunked_documents.json: {e}")
        print("Creating chunked_documents.json from dataset...")

        # Download and extract the textbooks if needed
        download_and_extract_zip(dataset_url)

        # Load, clean, and process documents
        documents = load_text_files(DATA_DIR / "en")
        cleaned_documents = [clean_and_tokenize(doc) for doc in documents]
        chunked_documents = []
        for doc in cleaned_documents:
            chunked_documents.extend(chunk_text(doc))
        print(f"Total document chunks created: {len(chunked_documents)}")

        # Save the chunked documents to JSON
        with open(CHUNKED_DOCUMENTS_PATH, "w", encoding="utf-8") as f:
            json.dump(chunked_documents, f)
        print("chunked_documents.json created.")

print(f"Total document chunks available: {len(chunked_documents)}")


In [ ]:
import os
import numpy as np
import faiss
from pathlib import Path
import torch
from transformers import AutoTokenizer, AutoModel

# Assuming 'chunked_documents' is already defined from earlier processing
INDEX_PATH = "./faiss_index.idx"
FAISS_INDEX_URL = "https://www.dropbox.com/scl/fi/05ez2886nz5fkkcqsv6hs/faiss_index.idx?rlkey=yil6ollju5smk04upluenqot4&st=yu0oji49&dl=1"
dimension = 384  # Embedding size from MiniLM model
# Load model and tokenizer (using PyTorch) only when needed
retrieval_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
retrieval_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

if os.path.exists(INDEX_PATH):
    print("Loading existing FAISS index from disk...")
    index = faiss.read_index(INDEX_PATH)
    print(f"Total embeddings indexed: {index.ntotal}")
else:
    print("FAISS index does not exist. Trying to download from remote URL...")
    try:
        response = requests.get(FAISS_INDEX_URL, allow_redirects=True)
        with open(INDEX_PATH, "wb") as f:
            f.write(response.content)
        print("Successfully downloaded FAISS index from remote URL.")
        index = faiss.read_index(INDEX_PATH)
        print(f"Total embeddings indexed: {index.ntotal}")
    except Exception as e:
        print("FAISS index not found. Creating index...")
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        retrieval_model.to(device)
        retrieval_model.eval()  # Set the model to evaluation mode

        # Function to generate embeddings for all chunks in a batch
        def get_embeddings_in_batch(texts, batch_size=16):
            all_embeddings = []

            # Wrap the loop with tqdm to display a progress bar
            for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
                batch_texts = texts[i:i + batch_size]

                # Tokenize the batch of texts
                inputs = retrieval_tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
                # Generate embeddings on the GPU
                outputs = retrieval_model(**inputs).last_hidden_state  # [batch_size, sequence_length, hidden_size]
                batch_embeddings = torch.mean(outputs, dim=1).cpu().detach().numpy()  # [batch_size, hidden_size]

                # Append batch embeddings to the list
                all_embeddings.extend(batch_embeddings)

            return np.array(all_embeddings)

        # Generate embeddings for all document chunks in batches
        embeddings = get_embeddings_in_batch(chunked_documents, batch_size=128)
        print(f"Generated embeddings for {len(embeddings)} document chunks.")

        # Create the FAISS index and add embeddings
        index = faiss.IndexFlatL2(dimension)
        # Ensure embeddings are in the correct shape and type
        embedding_matrix = np.array([embedding.flatten() for embedding in embeddings]).astype('float32')
        index.add(embedding_matrix)
        print(f"Total embeddings indexed: {index.ntotal}")

        # Write the FAISS index to disk
        faiss.write_index(index, INDEX_PATH)
        print(f"FAISS index written to {INDEX_PATH}")


## Retrival Method

In [ ]:
retrieval_model.cpu().eval()

# Function to generate embeddings for a new query
def get_query_embedding(query):
    pass

# Load FAISS index with existing embeddings
embedding_dim = 384
index = faiss.IndexFlatL2(embedding_dim)

# Function to retrieve relevant documents based on the query
def retrieve_documents(query, top_k=5):
    pass

# Test retrieval component
sample_query = "What are the symptoms of heart failure?"
similar_documents = retrieve_documents(sample_query)
print("Retrieved documents:", similar_documents)


## Generation Method

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generation_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)
generation_model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", device_map="cuda", torch_dtype="auto", trust_remote_code=True)
generation_model.to(device).eval()


In [ ]:

# Function to generate a response using retrieved context
def generate_response(query, context, max_new_tokens=100):
    input_text = None

    # Tokenize the input and move tensors to GPU
    inputs = None

    # Generate response using max_new_tokens to control output length
    None

    # Decode the generated response
    response_text = generation_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response_text

# Testing generation with retrieved documents as context
retrieved_text = " ".join(similar_documents)  # Concatenate retrieved documents as context
response = generate_response(sample_query, retrieved_text)
print("Generated response:", response)